Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [ ]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [ ]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [ ]:

import numpy as np
from collections import namedtuple
from heapq import heappop, heappush
from random import choice
from tqdm import tqdm

# Solvability check
def is_solvable(state: np.ndarray) -> bool:
    flat_state = state.flatten()
    flat_state = flat_state[flat_state != 0]  # Exclude the empty tile
    inversions = sum(1 for i in range(len(flat_state)) for j in range(i + 1, len(flat_state)) if flat_state[i] > flat_state[j])
    if PUZZLE_DIM % 2 == 1:
        return inversions % 2 == 0
    else:
        empty_row = PUZZLE_DIM - np.where(state == 0)[0][0]
        return (inversions + empty_row) % 2 == 0
    
# Create a solvable puzzle from a non solvable one
def make_solvable(state: np.ndarray) -> np.ndarray:
    flat_state = state.flatten()
    flat_state = flat_state[flat_state != 0]  # Exclude the empty tile
    inversions = sum(1 for i in range(len(flat_state)) for j in range(i + 1, len(flat_state)) if flat_state[i] > flat_state[j])
    empty_row = PUZZLE_DIM - np.where(state == 0)[0][0]
    if inversions % 2 == 0:
        flat_state[-2], flat_state[-1] = flat_state[-1], flat_state[-2]
    else:
        flat_state[-3], flat_state[-4] = flat_state[-4], flat_state[-3]
    new_state = state.copy()
    new_state[new_state != 0] = flat_state
    return new_state

    def search(state, g, threshold, path):
        """Performs a bounded search up to the given threshold."""
        f = g + heuristic(state, goal_state)
        if f > threshold:
            return f, None
        if np.array_equal(state, goal_state):
            return True, path

        min_threshold = float('inf')
        for act in available_actions(state):
            next_state = do_action(state, act)
            if len(path) > 0 and path[len(path) - 1].pos1 == act.pos2:
                continue  # Avoid undoing the last move
            new_g = g + 1
            result, new_path = search(next_state, new_g, threshold, path + [act])
            if result is True:
                return True, new_path
            if result < min_threshold:
                min_threshold = result

        return min_threshold, None

    threshold = heuristic(initial_state, goal_state)
    while True:
        result, path = search(initial_state, 0, threshold, [])
        if result is True:
            return path
        if result == float('inf'):
            return "No solution found."
        threshold = result

Various heuristics


In [ ]:
# Manhattan distance heuristic
def manhattan(state: np.ndarray, goal: np.ndarray) -> int:
    distance = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            value = state[i, j]
            if value != 0:
                target_x, target_y = divmod(np.where(goal == value)[0][0], PUZZLE_DIM)
                distance += abs(i - target_x) + abs(j - target_y)
    return distance

def hamming(state: np.ndarray, goal: np.ndarray) -> int:
    return np.sum(state != goal) - 1  # Exclude the empty tile

import numpy as np

def linear_conflict_manhattan(state: np.ndarray, goal: np.ndarray) -> int:
    def linear_conflict(state, goal):
        """Compute linear conflicts."""
        conflict_count = 0
        # Check rows
        for row in range(PUZZLE_DIM):
            tiles_in_row = [state[row, col] for col in range(PUZZLE_DIM) if state[row, col] != 0]
            goal_positions = {tile: divmod(np.where(goal == tile)[0][0], PUZZLE_DIM)[1] for tile in tiles_in_row}
            for i, tile1 in enumerate(tiles_in_row):
                for tile2 in tiles_in_row[i + 1:]:
                    if goal_positions[tile1] > goal_positions[tile2]:
                        conflict_count += 1
        # Check columns
        for col in range(PUZZLE_DIM):
            tiles_in_col = [state[row, col] for row in range(PUZZLE_DIM) if state[row, col] != 0]
            goal_positions = {tile: divmod(np.where(goal == tile)[0][0], PUZZLE_DIM)[0] for tile in tiles_in_col}
            for i, tile1 in enumerate(tiles_in_col):
                for tile2 in tiles_in_col[i + 1:]:
                    if goal_positions[tile1] > goal_positions[tile2]:
                        conflict_count += 1
        return 2 * conflict_count  # Each conflict adds 2 moves

    # Combine Manhattan distance and linear conflict
    return manhattan(state, goal) + linear_conflict(state, goal)


Algos

In [ ]:
# A*  or Dijkstra's search

def search_astar(initial_state: np.ndarray, goal_state: np.ndarray, heuristic, use_heuristic: bool = True):
    """
    Unified search algorithm for A* and Dijkstra's based on the use_heuristic flag.
    
    Args:
        initial_state (np.ndarray): Initial puzzle state.
        goal_state (np.ndarray): Goal puzzle state.
        use_heuristic (bool): If True, uses A*; if False, uses Dijkstra's algorithm.
    
    Returns:
        list: Sequence of actions leading to the goal state, or an error message if unsolvable.
    """
    if not is_solvable(initial_state):
        return "Puzzle is not solvable."

    visited = set()
    priority_queue = []
    heappush(priority_queue, (0, 0, initial_state.tobytes(), []))  # (priority, cost, state, path)
    steps = 0

    while priority_queue:
        _, cost, current_state_bytes, path = heappop(priority_queue)
        current_state = np.frombuffer(current_state_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)

        if np.array_equal(current_state, goal_state):
            #print cost and steps
            print(f"Steps: {steps}, Cost: {cost}")
            return path

        visited.add(current_state_bytes)

        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            new_state_bytes = new_state.tobytes()
            if new_state_bytes not in visited:
                new_cost = cost + 1
                # Priority is cost + heuristic for A*, or just cost for Dijkstra's
                priority = new_cost + (heuristic(new_state, goal_state) if use_heuristic else 0)
                heappush(priority_queue, (priority, new_cost, new_state_bytes, path + [act]))

        steps += 1
        if steps % 100_000 == 0:
            print(f"Steps: {steps}, Cost: {cost}")
        if steps > 1_000_000:
            #Print solution and return
            print("Final state: \n", current_state)
            print(f"Steps: {steps}, Cost: {cost}")
            return path

    return "No solution found."



## Create solvable puzzle


In [ ]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
RANDOMIZE_STEPS = 100_000
state = goal_state.copy()
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

while not is_solvable(state):
    state = make_solvable(state)

print("Solvable puzzle generated.")


In [ ]:
# Which algorithm to run?
RUN = 'Dijkstra'  # 'A*' or 'Dijkstra'
# Solve the puzzle
switch = {
    'A*': search_astar,
    'Dijkstra': search_astar
}
use_heuristic = True
if RUN == 'Dijkstra':
    use_heuristic = False

HEURISTIC = 'linear_conflict_manhattan'
switch_heuristic = {
    'manhattan': manhattan,
    'hamming': hamming,
    'linear_conflict_manhattan': linear_conflict_manhattan
}
heuristic = switch_heuristic[HEURISTIC]

# Test for state 2 8 7 3 6 5 0 1 4
state = np.array([[2, 8, 7], [3, 6, 5], [0, 1, 4]])


solve_puzzle = switch[RUN]
solution = solve_puzzle(state, goal_state, heuristic, HEURISTIC)
print("Initial state:")
print(state)
print("\nSolution steps:")
if isinstance(solution, list):
    for act in solution:
        print(act.pos1, '->', act.pos2)
else:
    print(solution)